# Submit CP2K Geometry Optimization

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.code import Code
from aiida.orm.calculation.job.cp2k import Cp2kCalculation
from aiida.orm import load_node
from aiida.work.workfunction import workfunction

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview

from collections import OrderedDict

## Step 1: Select Energy Calculation from Database

In [ ]:
# query AiiDA database for energy calculations
qb = QueryBuilder()
qb.append(ParameterData, tag="params", filters={'attributes.GLOBAL.RUN_TYPE':{'ilike':'energy'}})
qb.append(Cp2kCalculation, tag="calc", output_of="params", project=["id", "description"])
qb.order_by({'calc': {'ctime': 'desc'}})
#print qb.count()

all_energy_calcs = OrderedDict()
all_energy_calcs["Select a Calculation"] = None
for pk, descr in qb.iterall():
    label = "PK: %d"%pk
    if descr:
        label += "; descr: "+descr
    all_energy_calcs[label] = pk

energy_calc = None
def on_energy_calc_change(c):
    global energy_calc
    energy_calc = load_node(drop_energy_calc.value)

drop_energy_calc = ipw.Dropdown(options=all_energy_calcs, value=None)
drop_energy_calc.observe(on_energy_calc_change, names='value')
display(drop_energy_calc)

## Step 2: Choose parameters

In [ ]:
rows = []
w = "100px"

# inp_compname = ipw.Dropdown(options=get_computer_names())
# rows.append(ipw.HBox([ipw.Label("Computer", width=w), inp_compname]))

# inp_xc = ipw.Dropdown(options=["LDA", "PBE"])
# rows.append(ipw.HBox([ipw.Label("XC Functional", width=w), inp_xc]))

# inp_basis = ipw.Dropdown(options=["SZV-MOLOPT-SR-GTH", "DZVP-MOLOPT-SR-GTH"])
# rows.append(ipw.HBox([ipw.Label("Basis Set", width=w), inp_basis]))

inp_descr = ipw.Text(placeholder="(optional)")
rows.append(ipw.HBox([ipw.Label("Description", width=w), inp_descr]))

display(ipw.VBox(rows))

In [ ]:
@workfunction
def build_parameters(energy_params):
    params = energy_params.get_attrs()
    params['GLOBAL']['RUN_TYPE'] = "geo_opt"
    return ParameterData(dict=params)

In [ ]:
def build_calc():
    code = energy_calc.inp.code
    energy_params = energy_calc.inp.parameters
    structure = energy_calc.inp.structure

    # calc object
    calc = code.new_calc()
    calc.description = inp_descr.value

    # structure
    calc.use_structure(structure)

    # parameters
    parameters = build_parameters(energy_params)
    calc.use_parameters(parameters)

    # resources
    calc.set_max_wallclock_seconds(3*60) # 3 min
    calc.set_resources({"num_machines": 1})
    return calc

## Step 3: Test submission

In [ ]:
def on_test(b):
    clear_output()
    if energy_calc is None:
        print("Please select an energy calculation.")
        return
    
    calc = build_calc()
    folder = calc.submit_test()[0]
    fn = folder.get_abs_path(calc._INPUT_FILE_NAME)
    content = open(fn).read()
    pre_tag = '<pre style="width:600px; max-height:250px; overflow-x:auto; line-height:1em; font-size:0.8em;">'   
    box_preview.value = pre_tag+content+"</pre>"
    # toggle visibility to workaround an apparent bug
    box_preview.visible = not chk_preview.value
    box_preview.visible = chk_preview.value
    print("Test went fine :-)")
    
btn_test = ipw.Button(description="Test")
btn_test.on_click(on_test)
chk_preview = ipw.Checkbox(description='preview', value=False)
box_preview = ipw.HTML()
ipw.jslink((chk_preview, 'value'), (box_preview, 'visible'))
display(ipw.VBox([ipw.HBox([btn_test, chk_preview]), box_preview]))

## Step 4: Submit calculation

In [ ]:
def on_submit(b):
    clear_output()
    calc = build_calc()
    # store and submit
    calc.store_all()
    calc.submit()
    msg = "Submitted calculation: UUID=%s, pk=%s .<br>"%(calc.uuid,calc.dbnode.pk)
    msg += 'Results can be viewed <a href="view_cp2k_calc.ipynb?pk=%d">here</a>'%calc.dbnode.pk
    display(HTML(msg))
    
btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
display(btn_submit)